In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re

In [2]:
newdata=pd.read_csv("../input/oathss/oaths.csv")

# Preprocessing

In [3]:
all_text=newdata["transcription_text"]

In [4]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()

In [5]:
stop_words.remove("of")
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punctuation:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

In [7]:
for i in tqdm(range(newdata.shape[0])):
    all_text[i] = process(all_text[i])

100%|██████████| 2136/2136 [00:06<00:00, 328.98it/s]


In [8]:
for i in tqdm(range(newdata.shape[0])):
    all_text[i] = " ".join(all_text[i])

100%|██████████| 2136/2136 [00:00<00:00, 3290.91it/s]


In [9]:
all_text

0       I I C Meekins of Currituck County State of Nor...
1       Department of Virginia North Carolina 1864 tak...
2       I W H Pearce of Craven County State of NorthCa...
3       State of North Carolina County I solemnly swea...
4       State of North Carolina Wake County I SE Allen...
                              ...                        
2131    Office Provost Marshal Raleigh NC May 26 65 Th...
2132    July 30th 1867 Birnie Wm Agent Bu R F A L Oath...
2133    State of North Carolina Robeson County I Willi...
2134    NARA 449 102 We whose name written well person...
2135    NARA 450 404 Office Provost Marshall City of W...
Name: transcription_text, Length: 2136, dtype: object

# Flair

In [10]:
!pip3 install Cython
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 37.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-p

# Hugging face  : Flair

**ner-english** 

In [11]:
!git lfs install
!git clone https://huggingface.co/flair/ner-english-large

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
Cloning into 'ner-english-large'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), 11.03 KiB | 1.22 MiB/s, done.


In [12]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-10 11:19:31,451 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-10 11:19:59,405 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [13]:
from flair.models import SequenceTagger

tagger1 = SequenceTagger.load("flair/ner-english")

Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2022-06-10 11:20:09,741 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-10 11:20:11,584 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [14]:
from flair.models import SequenceTagger

tagger2 = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-10 11:21:02,073 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-06-10 11:21:20,603 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


tagger3 nerenglishlarge

In [15]:
listenglarge=[]
label=[]
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dflarge=pd.DataFrame({"text":"", "PERSON-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in range(2136):
        sentencess =Sentence(all_text[i])
        dflarge.loc[i,"text"]=all_text[i]
        tagger2.predict(sentencess)
        dc={}
        P=[]
        L=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        listenglarge.append(entity)
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
            

dflarge["PERSON-nerenglarge"]=Per
dflarge["LOC-nerenglarge"]=LOC
dflarge["ORG-nerenglarge"]=ORG                        

In [16]:
dflarge

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,I I C Meekins of Currituck County State of Nor...,"[I I C Meekins, God, Isaac C Meekins, E H Walk...","[Currituck County State of North Carolina, Uni...",[]
1,Department of Virginia North Carolina 1864 tak...,[],"[North Carolina, United States, States]",[Department of Virginia]
2,I W H Pearce of Craven County State of NorthCa...,"[W H Pearce, Almighty, God, Jm E Ampetts, A B ...","[Craven County State of NorthCarolina, United ...",[]
3,State of North Carolina County I solemnly swea...,[God],"[State of North Carolina County, United States]",[]
4,State of North Carolina Wake County I SE Allen...,"[SE Allen, God SE Allen Sworn, McAlpine]","[State of North Carolina, Wake County, United ...",[Res Corp]
...,...,...,...,...
2131,Office Provost Marshal Raleigh NC May 26 65 Th...,"[Geo Little, Geo B Ayer, DH Grave, Hugo Hilleb...","[Raleigh, NC, Wake County State of NC, United ...",[]
2132,July 30th 1867 Birnie Wm Agent Bu R F A L Oath...,[],[],[]
2133,State of North Carolina Robeson County I Willi...,"[William Birnie, God, Wm Birnie, Dick JP]","[Robeson County, United States]",[]
2134,NARA 449 102 We whose name written well person...,"[AL Price, James Fulton, Fulton Price, Sgd W N...","[State of North Carolina, New Hanover County, ...","[NARA, Confederate Government, Court of Pleas,..."


tagger 2 ontonotes

In [17]:
listoto=[]
label=[]
Per=[]
Date=[]
GPE=[]
ORG=[]
from flair.data import Sentence

aldf=pd.DataFrame({"text":"", "PERSON-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})
for i in range(2136):
        sentencess =Sentence(all_text[i])
        aldf.loc[i,"text"]=all_text[i]
        tagger.predict(sentencess)
        dc={}
        P=[]
        D=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        listoto.append(entity)
        
        
        for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
        Per.append(P)
        Date.append(D)
        ORG.append(O)
        GPE.append(G)
            
aldf["Date-ontonotes"]=Date
aldf["PERSON-ontonotes"]=Per
aldf["GPE-ontonotes"]=GPE
aldf["ORG-ontonotes"]=ORG    

In [18]:
aldf.shape

(2136, 5)

In [19]:
aldf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes
0,I I C Meekins of Currituck County State of Nor...,"[I I C Meekins, Isaac C Meekins, E H Walker JP...",[North Carolina],[],[16 day of Sept AD 1865]
1,Department of Virginia North Carolina 1864 tak...,[],"[North Carolina, United States]",[Department of Virginia],[1864]
2,I W H Pearce of Craven County State of NorthCa...,"[W H Pearce, Watson]",[],[],[3rd day of October AD 1865]
3,State of North Carolina County I solemnly swea...,[],"[North Carolina County, United States]",[],[]
4,State of North Carolina Wake County I SE Allen...,[Allen],"[North Carolina, Wake County, United States]",[],[31st day of May 1867]


In [20]:
aldf.shape

(2136, 5)

# Post processing

**For dates**

In [21]:
G=[]
for i in aldf["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-ZZa-z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)

    
aldf["postDate-ontonotes"]=GF
            
            
        

# Run this to see the extracted dates before filtring them 

In [22]:
#GF

In [23]:
G=[]
for i in aldf["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        final_date4=re.sub(r'(^[A-Za-z]\S*.*.)day\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
        if final_date4 != j :
            M.append(final_date4)
    G.append(M)
    

aldf["postDate-ontonotes"]=G

**great !**

****

****

In [24]:
dflarge.head()

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,I I C Meekins of Currituck County State of Nor...,"[I I C Meekins, God, Isaac C Meekins, E H Walk...","[Currituck County State of North Carolina, Uni...",[]
1,Department of Virginia North Carolina 1864 tak...,[],"[North Carolina, United States, States]",[Department of Virginia]
2,I W H Pearce of Craven County State of NorthCa...,"[W H Pearce, Almighty, God, Jm E Ampetts, A B ...","[Craven County State of NorthCarolina, United ...",[]
3,State of North Carolina County I solemnly swea...,[God],"[State of North Carolina County, United States]",[]
4,State of North Carolina Wake County I SE Allen...,"[SE Allen, God SE Allen Sworn, McAlpine]","[State of North Carolina, Wake County, United ...",[Res Corp]


In [25]:
aldf["Person-NerEngLarge"]=dflarge["PERSON-nerenglarge"]
aldf["LOC-NerEngLarge"]=dflarge["LOC-nerenglarge"]
aldf["ORG-NerEngLarge"]=dflarge["ORG-nerenglarge"]


In [26]:
allPerson=[]
for i in aldf["PERSON-ontonotes"]:
            allPerson.append(i)
allDatess=[]
for i in aldf["Date-ontonotes"]:
            allDatess.append(i)
allpostdates=[]
for i in aldf["postDate-ontonotes"]:
            allpostdates.append(i)
allGPE=[]
for i in aldf["GPE-ontonotes"]:
            allGPE.append(i)
alORGl=[]
for i in aldf["ORG-ontonotes"]:
            alORGl.append(i)

****

In [27]:
Personlarge=[]
for i in dflarge["PERSON-nerenglarge"]:
            Personlarge.append(i)
ORGlarge=[]
for i in dflarge["ORG-nerenglarge"]:
            ORGlarge.append(i) 
LOClarge=[]
for i in dflarge["LOC-nerenglarge"]:
            LOClarge.append(i)

* as usual , ner English Large is better in extracting Person 
* ontonotes is thes best in extracting dates after good postprocessing and filtring almost all pattern 
* ner Engish Large  also better in Location and Oranization 

# dates

**run those cells to take a look at the result more clearly**

In [28]:
#G

# Person

In [29]:
 # Personlarge

# LOC and ORG 

In [30]:
# LOClarge

In [31]:
allGPE[1000:1020]

[['United States of America', 'Charleston SC'],
 ['United States of America', 'South Carolina', 'Charleston SC'],
 ['United States of America', 'Charleston'],
 ['Charleston', 'Charleston SC', 'United States'],
 ['Charleston', 'United States of America', 'Charleston SC'],
 ['United States of America', 'Blackville'],
 ['United States of America', 'South Carolina', 'United States', 'Blackville'],
 ['United States of America', 'Blackville'],
 ['United States of America', 'Charleston SC', 'Charleston', 'South Carolina'],
 ['United States of America', 'Charleston SC'],
 ['United States of America',
  'NO UNITED STATES OF AMERICA',
  'Charlestown',
  'South Carolina'],
 ['United States of America', 'South Carolina'],
 ['United States of America',
  'UNITED STATES OF AMERICA',
  'Charleston District',
  'South Carolina'],
 ['United States of America', 'Charleston SC'],
 ['United States Of America',
  'Charleston SC',
  'UNITED STATES OF America',
  'Charleston'],
 ['South Carolina'],
 ['United

In [32]:
all_text[1020]

'State of South Carolina I James E Thuring of Barnwell District solemnly swear I carefully read amnesty proclamation issued Andrew Johnson President of United States of America May 29 1865 I excepted benefit Proclamation one of fourteen exception therein made J E Thuring Sworn subscribed 23d day of November AD 1865 Aiken SC Fredk A Ford Magistrate Barnwell Dist SC'

In [33]:
LOClarge[1200]

['United States of America',
 'Charleston',
 'SC',
 'Unites States of America',
 'City of Charleston',
 'State of South Carolina',
 'United States Union of States']

# the final dataset :

In [34]:
Personlarge[:10]

[['I I C Meekins',
  'God',
  'Isaac C Meekins',
  'E H Walker',
  'I C Meekins',
  'John B Etheridge'],
 [],
 ['W H Pearce', 'Almighty', 'God', 'Jm E Ampetts', 'A B Watson'],
 ['God'],
 ['SE Allen', 'God SE Allen Sworn', 'McAlpine'],
 ['Sarah E Dough', 'God', 'Isaac C Merkins'],
 ['Sarah E Dough', 'JM Wilson'],
 ['I', 'W T Dough', 'God', 'E H Walker', 'John B Etheridge'],
 ['Jesse E Dough', 'Almighty', 'God', 'Isaac C Meekins', 'A D Etheridge'],
 ['I T A Dough', 'God', 'John B Etheridge', 'T A Dough']]

In [35]:
aldf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Person-NerEngLarge,LOC-NerEngLarge,ORG-NerEngLarge
0,I I C Meekins of Currituck County State of Nor...,"[I I C Meekins, Isaac C Meekins, E H Walker JP...",[North Carolina],[],[16 day of Sept AD 1865],[16-Sept-1865],"[I I C Meekins, God, Isaac C Meekins, E H Walk...","[Currituck County State of North Carolina, Uni...",[]
1,Department of Virginia North Carolina 1864 tak...,[],"[North Carolina, United States]",[Department of Virginia],[1864],[],[],"[North Carolina, United States, States]",[Department of Virginia]
2,I W H Pearce of Craven County State of NorthCa...,"[W H Pearce, Watson]",[],[],[3rd day of October AD 1865],[3-October-1865],"[W H Pearce, Almighty, God, Jm E Ampetts, A B ...","[Craven County State of NorthCarolina, United ...",[]
3,State of North Carolina County I solemnly swea...,[],"[North Carolina County, United States]",[],[],[],[God],"[State of North Carolina County, United States]",[]
4,State of North Carolina Wake County I SE Allen...,[Allen],"[North Carolina, Wake County, United States]",[],[31st day of May 1867],[31-May-1867],"[SE Allen, God SE Allen Sworn, McAlpine]","[State of North Carolina, Wake County, United ...",[Res Corp]


# list of ontonotes entity

# abdesslem  please tell me if those lists i will pickle them ? because like that i won't know which one belongs to the first or the second text ??

# Pickle

In [36]:
# import pickle
# my_pickled_list = pickle.dumps(listoto)  # Pickling the object
# print(f"This is my pickled object:\n{my_pickled_list}\n")

# my_pickled_list

# my_unpickled_object = pickle.loads(my_pickled_list)  # Unpickling the object
# print(
#     f"This is a_dict of the unpickled object:\n{my_unpickled_object}\n")


# my_unpickled_object

In [37]:
# L=[]
# for i in G:
#        for j in range(len(i)):
#             while i[j]!="-":
#                 L.append(i[j])
#             j=j+1
        

In [38]:
# if i[1] in ["August","Aug","augt","AUGUST"]:
#         i[1]= "August"
#     elif i[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER"]:
#         i[1]= "Septembre"
#     elif i[1] in ["October","Oct","OCT","OCTOBER"]:
#         i[1]= "October"
#     elif i[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER"]:
#         i[1]= "November"
#     elif i[1] in ["December","Dec","Decbr","DEC","DECEMBER"]:
#         i[1]= "December"
#     elif i[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
#         i[1]= "January"
#     elif i[1] in ["February","Feby","Febry","FEB","FEBRUARY"]:
#         i[1]= "February"
#     elif i[1] in ["March","Mar"]:
#         i[1]= "March"
#     elif i[1] in ["April","Apl","Aprl","APRIL"]:
#         i[1]= "April"
#     elif i[1] in ["May","MAY"]:
#         i[1]= "May"
#     elif i[1] in ["JUN","Jun","June"]:
#         i[1]= "June"
#     elif i[1] in ["JULY","July","JUL"]:
#         i[1]= "July"
    

In [39]:
# date=re.findall(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',"21st Aug 2022")
# dates=[]
# for i in date:
#     for j in i:
#         dates.append(j)

In [40]:
# if dates[1] in ["August","Aug","augt","AUGUST"]:
#         dates[1]= "August"

In [41]:
# ch=["21st Sep 2022", "33 Aug 2022"]
# nL=[]
# for i in ch:
#     L=[]
#     datee=re.findall(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',i)
#     dates=[]
#     for i in datee:
#         for j in i:
#             dates.append(j)
        
        
#         if dates[1] in ["August","Aug","augt","AUGUST"]:
#             dates[1]= "August"
#         elif dates[1] in ["September","Sept","SEP","Sep","sep","Septr","SEPTEMBER"]:
#             dates[1]= "Septembre"
   
        
#         ch = dates[0]+"-"+dates[1]+"-"+dates[2]

#         L.append(ch)
#     nL.append(L)
    

In [42]:
date=re.findall(r'([0-9]{1,2})-([A-Z][a-z]\S*)-([0-9]{2,4})',"twenty-July-1865")
date
if date!=[]:
    print("hello")
else :
    date = re.findall(r'(^[A-Za-z]\S*.*.)-([A-Z][a-z]\S*)-([0-9]{2,4})', "twenty-July-1865")
    print("helo")

    if date!=[]:
        print(date)
    

helo
[('twenty', 'July', '1865')]


In [43]:
Gi=[["fifteenth-Aug-2022"],["15-Sep-2022"],["eight-Aug-2022"]]
NG=[]
for i in Gi:
    ng=[]
    for j in i:
        
        date=re.findall(r'([0-9]{1,2})-([A-Z][a-z]\S*)-([0-9]{2,4})',j)
        if (date !=[]):
            dates=[]
            for k in date:
                for l in k:
                    dates.append(l)
                
                
                if dates[1] in ["August","Aug","augt","AUGUST"]:
                    dates[1]= "August"
                elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER"]:
                    dates[1]= "Septembre"
                j = dates[0]+"-"+dates[1]+"-"+dates[2]
                ng.append(j)
        else :
            date = re.findall(r'(^[A-Za-z]\S*.*.)-([A-Z][a-z]\S*)-([0-9]{2,4})',j )
            dates=[]
            for k in date:
                for l in k:
                    dates.append(l)

                   
                
                
                if dates[1] in ["August","Aug","augt","AUGUST"]:
                    dates[1]= "August"
                elif dates[1] in ["JUN","Jun","June"]:
                    dates[1]="Jun"
                if dates[0]== "fifteenth" :
                    dates[0]="15"
                elif dates[0]== "eight" :
                    dates[0]="8"
                j = dates[0]+"-"+dates[1]+"-"+dates[2]
                ng.append(j)
            
            
    NG.append(ng)   

# standardizing dates

In [44]:
NG=[]
for i in G:
    ng=[]
    for j in i:
        
        date=re.findall(r'([0-9]{1,2})-([A-Z][a-z]\S*)-([0-9]{2,4})',j)
        if date !=[]:
            dates=[]
            for k in date:
                for l in k:
                    dates.append(l)
                
                
                if dates[1] in ["August","Aug","augt","AUGUST","Augst","Augt","AUG","Augusta"]:
                    dates[1]= "August"
                elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER",'Sepr','Septh']:
                    dates[1]= "Septembre"
                elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                    dates[1]= "October"
                elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER"]:
                    dates[1]= "November"
                elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                    dates[1]= "December"
                elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                    dates[1]= "January"
                elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY"]:
                    dates[1]= "February"
                elif dates[1] in ["March","Mar"]:
                    dates[1]= "March"
                elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                    dates[1]= "April"
                elif dates[1] in ["May","MAY"]:
                    dates[1]= "May"
                elif dates[1] in ["JUN","Jun","June"]:
                    dates[1]= "June"
                elif dates[1] in ["JULY","July","JUL"]:
                    dates[1]= "July"
                j = dates[0]+"-"+dates[1]+"-"+dates[2]
                ng.append(j)
        else :
                date = re.findall(r'(^[A-Za-z]\S*.*.)-([A-Z][a-z]\S*)-([0-9]{2,4})',j ) 
                dates=[]
                for k in date:
                    for l in k:
                        dates.append(l)
                
                
                    if dates[1] in ["August","Aug","augt","AUGUST","Augst",'Augt',"AUG","Augusta"]:
                        dates[1]= "August"
                    elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER","Sepr","Septh", "This"]:
                        dates[1]= "Septembre"
                    elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                        dates[1]= "October"
                    elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER"]:
                        dates[1]= "November"
                    elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                        dates[1]= "December"
                    elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                        dates[1]= "January"
                    elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY"]:
                        dates[1]= "February"
                    elif dates[1] in ["March","Mar"]:
                        dates[1]= "March"
                    elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                        dates[1]= "April"
                    elif dates[1] in ["May","MAY"]:
                        dates[1]= "May"
                    elif dates[1] in ["JUN","Jun","June"]:
                        dates[1]= "June"
                    elif dates[1] in ["JULY","July","JUL"]:
                        dates[1]= "July"
                    if dates[0]== "fifteenth" :
                        dates[0]="15"
                    elif dates[0]=="sixteenth":
                        dates[0]="16"
                    elif dates[0]=="Twenty":
                        dates[0]="20" 
                    elif dates[0]=="Twentyseventh":
                        dates[0]="27" 
                    elif (dates[0]=="Twenty Eighth") |(dates[0]=="TwentyEight") |(dates[0]=="TwentyEight ") |(dates[0]=="Twentyeight"):
                        dates[0]="28"
                    elif (dates[0]=="Ninth") |(dates[0]=="Ninth ") |(dates[0]=="ninth") |(dates[0]=="ninth ")|(dates[0]=="Augusta 9th"):
                        dates[0]="9"
                    
                    elif dates[0]=='Seventh':
                        dates[0]="7" 
                    elif (dates[0]=="Eighth ") |(dates[0]=="Eighth") |(dates[0]=="eighth") |(dates[0]=="eighth "):
                        dates[0]="8"
                    elif dates[0]=="Seventh ":
                        dates[0]="7"
                    elif (dates[0]=="Twenty ninth ")| (dates[0]=="Twenty ninth") | (dates[0]=="twenty ninth ")|(dates[0]=="Twenty Ninth ")|(dates[0]=="Twenty Ninth") :
                        dates[0]="29"
                    elif (dates[0]=="Thirty") |(dates[0]=="Thirty "):
                        dates[0]="30"
                    elif (dates[0]=="Thirty first ") |(dates[0]=="Thirty first"):
                        dates[0]="31"
                  
                    
                    elif dates[0]=="first ":
                        dates[0]="1"
                    elif (dates[0]=="Second ") |(dates[0]=="Second") |(dates[0]=="second")|(dates[0]=="second "):
                        dates[0]="2"
                    elif dates[0]=="fourth ":
                        dates[0]="4"
                    elif dates[0]=="tenth ":
                        dates[0]="10"
                    elif (dates[0]=="Fifth ")|(dates[0]=="Fifth") |(dates[0]=="fifth ")|(dates[0]=="fifth"):
                        dates[0]="5"
                    elif (dates[0]=="This") | (dates[0]=="This 7th ")|(dates[0]=="seventh ") |(dates[0]=="seventh") |(dates[0]=="Seventh "):
                        dates[0]="7"
                    elif (dates[0]=="This 31st") |(dates[0]=="Thirty-first "):
                        dates[0]="31"
                    elif (dates[0]=="Thirtieth") |(dates[0]=="Thirtieth ") |(dates[0]=="Thirty"):
                        dates[0]="30"
                    
                    elif dates[0]=="Twenty seventh ":
                        dates[0]="27"
                    elif (dates[0]=="Twenty sixth ") |(dates[0]=="Twentysixth ")|(dates[0]=="Twentysixth") |(dates[0]=="twentysixth "):
                        dates[0]="26"
                    elif dates[0]=="third ":
                        dates[0]="3" 
                    elif dates[0]=="third":
                        dates[0]="3"
                    elif dates[0]=="Third ":
                        dates[0]="3"
                    elif dates[0]=="Third":
                        dates[0]="3"
                    
                    elif dates[0]=="Tenth":
                        dates[0]="10" 
                    elif (dates[0]=="this_sixth ")|(dates[0]=="sixth ")| (dates[0]=="Sixth") | (dates[0]=="Sixth "):
                        dates[0]="6" 
                        
                    elif dates[0]=="Tenth ":
                        dates[0]="10"
                    elif (dates[0]=="twentieth")|(dates[0]=="This 20th "):
                        dates[0]="20"
                    elif (dates[0]=="twenty fourth ") |(dates[0]=="twenty fourth") |(dates[0]=="Twenty fourth ") |(dates[0]=="Twenty fourth"):
                        dates[0]="24"
                    elif (dates[0]=="fifteenth ") |(dates[0]=="Fifteenth ") |(dates[0]=="Fifteenth"):
                        dates[0]="15"
                    elif dates[0]=="twentieth ":
                        dates[0]="20"
                    elif dates[0]=="Eighteenth ":
                        dates[0]="18"
                    elif (dates[0]=="Nineteenth ") |(dates[0]=="Nineteenth"):
                        dates[0]="18"
                    elif dates[0]=="First ":
                        dates[0]="1"
                    elif dates[0]=="First":
                        dates[0]="1"
                    elif (dates[0]=="Sixteenth ")|(dates[0]=="this 16") |(dates[0]=="Sixteenth"):
                        dates[0]="16"
                   
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif dates[0]=="twenty third":
                        dates[0]="23"
                    elif dates[0]=="Twenty Second ":
                        dates[0]="22"
                    elif dates[0]=="Twenty Second":
                        dates[0]="22"
                    elif dates[0]=="Twenty Eighth ":
                        dates[0]="28"
                    elif dates[0]=="twenty third":
                        dates[0]="23"
                    elif (dates[0]=="twenty third") |(dates[0]=="Twentythird")|(dates[0]=="Twentythird "):
                        dates[0]="23"
                    elif (dates[0]=="fourteenth ") | (dates[0]=="fourteenth") | (dates[0]=="Fourteenth") | (dates[0]=="Fourteenth "):
                        dates[0]="14"
                    elif (dates[0]=="Eleventh ") | (dates[0]=="Eleventh") |(dates[0]=="eleventh ") :
                        dates[0]="11"
                    elif (dates[0]=="Thirteenth") | (dates[0]=="Thirteenth ") :
                        dates[0]="13"
                    elif (dates[0]=="senventeenth ") |(dates[0]=="senventeenth") :
                        dates[0]="17"
                     
                     
                     
                    
                        
                    j = dates[0]+"-"+dates[1]+"-"+dates[2]
                    ng.append(j)
            
            
    NG.append(ng)   

In [45]:
NG[:500]

[['16-Septembre-1865'],
 [],
 ['3-October-1865'],
 [],
 ['31-May-1867'],
 ['1-December-1866'],
 ['30-May-1864'],
 ['16-Septembre-1865'],
 ['16-November-1866'],
 ['16-Septembre-1865'],
 ['4-November-1867'],
 ['4-November-1867'],
 ['2-October-1865'],
 [],
 ['31-December-1867'],
 ['19-June-1865', '19-June-1865'],
 ['4-November-1865'],
 ['9-November-1866',
  '9-The-1866',
  'The-November-1866',
  'The 9th -November-1866'],
 ['1-December-1866', '1-December-1866'],
 ['25-June-1864'],
 ['20-October-1866'],
 ['3-December-1866', '3-December-1866'],
 ['20-July-1867', '24-July-1867'],
 ['19-May-1865', '18-May-1865', '18-May-1865'],
 [],
 ['20-October-1866'],
 ['15-January-1868'],
 ['1-July-1867'],
 [],
 [],
 [],
 ['3-August-67'],
 ['3-August-1867'],
 ['15-December-67'],
 [],
 [],
 [],
 ['15-April-67'],
 [],
 ['23-July-67'],
 ['23-July-1867'],
 ['26-July-1867'],
 ['25-July-1867'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['29-August-66'],
 [],
 ['1-November-1866'],
 [],
 ['9-November-66'],
 [],
 [],
 ['26-J